In [2]:
# минимально необходимые внешние модули
#!pip install chardet
#!pip install dawg
#!pip install gensim
#!pip install pattern
#!pip install razdel
#!pip install pymorphy2

import sys, os
# сменим рабочую директорию на ту, где расположен каталог с библиотекой nlptk
os.chdir(r'D:\INSTALL\Python3\PROJECTS\REPOS\nlp_toolkit')

In [2]:
#from nlptk.mining.utils import datapath

In [3]:
# добавим этот путь в sys.path, чтобы интерпретатор увидел нашу библиотеку
sys.path.append(r'D:\INSTALL\Python3\PROJECTS\REPOS\nlp_toolkit')


In [4]:
import sys
from pprint import pprint
import nlptk
from nlptk.mining.text import *



In [5]:
prep = Prep()
prep.__dict__

{'sentencizer': <function nlptk.misc.mixins.SentencizerMixin.sentencize_nltk(text, *args, lang='english', **kwargs)>,
 'tokenizer': functools.partial(<function TokenizerMixin.toktok_tokenize at 0x12FCE390>, strip='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'),
 'tagger': <function nlptk.misc.mixins.TaggerMixin.tagger_nltk(tokens, *args, lang='eng', **kwargs)>,
 'lemmatizer': functools.partial(<function LemmatizerMixin.lemmatize_nltk at 0x12FCE150>, pos=True, normalize_uppercase=<method 'capitalize' of 'str' objects>)}

In [6]:

rules_clean = OrderedDict(
        roman_numerals=(True,)
)
    
rules_filter = OrderedDict(
        by_tagpos=(True,{},{"FW","POS"}), # фильтр по тегам частей речи - игнорировать иностранные слова и possessive ending parent's
        punctuation=True, # по умолчанию
        short=(True,3),   # по умолчанию
        stopwords=True,   # по умолчанию
        ifnotin_lexicon=True
        #trailing_chars=True
)

clean = TextCleaner(rules_clean)
filters = TokenFilter(rules_filter)   
    
        

In [8]:
filters.rules

In [20]:
clean.rules

OrderedDict([('hyphenation', (True,)),
             ('accent', (True,)),
             ('tags', (True,)),
             ('urls', (True,)),
             ('numeric', (True,)),
             ('nonletter_sequences', (True,)),
             ('quotes', (True,)),
             ('multiple_whitespaces', (True,)),
             ('roman_numerals', (True,))])

In [24]:
intake='''THE PICTURE OF DORIAN GRAY

BY

OSCAR WILDE. The artist is the creator of beautiful things. PREFACE'''
text1 = Text(intake, prep, clean, filters, input='text', inplace=True)
print(text1)

THE⁄DT⁄The PICTURE⁄NN⁄Picture OF⁄IN⁄Of DORIAN⁄JJ⁄Dorian GRAY⁄NNP⁄Gray BY⁄IN⁄By OSCAR⁄NNP⁄Oscar WILDE⁄NNP⁄Wilde
The⁄DT⁄The artist⁄NN⁄artist is⁄VBZ⁄be the⁄DT⁄the creator⁄NN⁄creator of⁄IN⁄of beautiful⁄JJ⁄beautiful things⁄NNS⁄thing
PREFACE⁄NN⁄Preface


In [7]:
text1.words(lower=False)

['PICTURE',
 'DORIAN',
 'GRAY',
 'OSCAR',
 'WILDE',
 'artist',
 'creator',
 'beautiful',
 'things']

In [9]:
text1.sents()

[TaggedSentence(
 	'THE⁄DT⁄The PICTURE⁄NN⁄Picture OF⁄IN⁄Of DORIAN⁄JJ⁄Dorian GRAY⁄NNP⁄Gray BY⁄IN⁄By OSCAR⁄NNP⁄Oscar WILDE⁄NNP⁄Wilde',
 	 n=0
 ), TaggedSentence(
 	'The⁄DT⁄The artist⁄NN⁄artist is⁄VBZ⁄be the⁄DT⁄the creator⁄NN⁄creator of⁄IN⁄of beautiful⁄JJ⁄beautiful things⁄NNS⁄thing',
 	 n=1
 )]

In [8]:
text1.sents(0).untagging()

[('THE', 'DT', 'The'),
 ('PICTURE', 'NN', 'Picture'),
 ('OF', 'IN', 'Of'),
 ('DORIAN', 'JJ', 'Dorian'),
 ('GRAY', 'NNP', 'Gray'),
 ('BY', 'IN', 'By'),
 ('OSCAR', 'NNP', 'Oscar'),
 ('WILDE', 'NNP', 'Wilde')]

In [45]:
intake = 'The Picture of Dorian Gray By Oscar Wilde'
text2 = Text(intake, prep, clean, filters, input='text', inplace=True)
print(text2)

The⁄DT⁄The Picture⁄NN⁄Picture of⁄IN⁄of Dorian⁄JJ⁄Dorian Gray⁄NNP⁄Gray By⁄IN⁄By Oscar⁄NNP⁄Oscar Wilde⁄NNP⁄Wilde


In [46]:
text2.sents(0).untagging()

[('The', 'DT', 'The'),
 ('Picture', 'NN', 'Picture'),
 ('of', 'IN', 'of'),
 ('Dorian', 'JJ', 'Dorian'),
 ('Gray', 'NNP', 'Gray'),
 ('By', 'IN', 'By'),
 ('Oscar', 'NNP', 'Oscar'),
 ('Wilde', 'NNP', 'Wilde')]

In [20]:
print(repr(text2))

Text(
	name='StringIO',
	nsents=1,
	nwords=8,
	nlemmas=8
)


In [9]:
source = os.path.abspath(os.path.join(MODULEDIR,r'corpus\en'))
corpus = Corpus(Path(source,"*.txt"), prep, clean, filters)
corpus.verbose = True
corpus.rewrite = True # перетегировать все тексты, даже если они сохранены 
#corpus.filtrate = True # применить фильтры при формировании общего вокабуляра; по умолчанию False
print(help(corpus.tfidf))


Help on method tfidf in module nlptk.mining.text:

tfidf(n_doc, token=None, texts: List[List[str]] = [], sort=False) method of nlptk.mining.text.Corpus instance

None


In [38]:
texts = []

for text in corpus:
    # можно выводить объекты текстов text и что-то с ними делать, только нужно помнить, что они - генераторы предложений
    # хотя параметр inplace=True заставит их отработать без цикла 
    texts.append(text)
    for i,sent in enumerate(text):
        pass # можно выводить предложения при обработке, но это замедлит работу 
        
print('Done!')


In [11]:
print(repr(corpus))

Corpus(
	names=['Austin Pride and Prejudice.txt', 'bronte_jane_txt.txt',...],
	ndocs=21,
	nwords=1927580,
	nlemmas=44991,
	nhapaxes=18047
)


In [9]:
# Последнее предложение в корпусе, которое осталось в этой переменной после цикла обработки
# сами предложения в объекте корпуса никак не сохраняются в целях экономии памяти. 
# Сохраняются только всевозможные частотные словари
sent

TaggedSentence(
	'THE⁄DT⁄The END⁄NN⁄End',
	 n=6503
)

In [12]:
# работаем с последним текстом, который интерпретатор сохранил в этой переменной
text

Text(
	name='wilde_picture_of_dorian_gray_txt.txt',
	nsents=13012,
	nwords=159588,
	nlemmas=5797
)

In [19]:
# объект text при печати на консоль отображается как текст аналогичный сохраненному на диске тегированному тексту
pprint(str(text).split("\n")[:10])

['THE⁄DT⁄The PICTURE⁄NN⁄Picture OF⁄IN⁄Of DORIAN⁄JJ⁄Dorian GRAY⁄NNP⁄Gray '
 'BY⁄IN⁄By OSCAR⁄NNP⁄Oscar WILDE⁄NNP⁄Wilde THE⁄DT⁄The PREFACE⁄NNP⁄Preface '
 'The⁄DT⁄The artist⁄NN⁄artist is⁄VBZ⁄be the⁄DT⁄the creator⁄NN⁄creator of⁄IN⁄of '
 'beautiful⁄JJ⁄beautiful things⁄NNS⁄thing',
 'To⁄TO⁄To reveal⁄VB⁄reveal art⁄NN⁄art and⁄CC⁄and conceal⁄VB⁄conceal '
 'the⁄DT⁄the artist⁄NN⁄artist is⁄VBZ⁄be art⁄JJ⁄art s⁄NN⁄s aim⁄NN⁄aim',
 'The⁄DT⁄The critic⁄NN⁄critic is⁄VBZ⁄be he⁄PRP⁄he who⁄WP⁄who can⁄MD⁄can '
 'translate⁄VB⁄translate into⁄IN⁄into another⁄DT⁄another manner⁄NN⁄manner '
 'or⁄CC⁄or a⁄DT⁄a new⁄JJ⁄new material⁄NN⁄material his⁄PRP$⁄his '
 'impression⁄NN⁄impression of⁄IN⁄of beautiful⁄JJ⁄beautiful things⁄NNS⁄thing',
 'The⁄DT⁄The highest⁄JJS⁄high as⁄IN⁄a the⁄DT⁄the lowest⁄JJS⁄low form⁄NN⁄form '
 'of⁄IN⁄of criticism⁄NN⁄criticism is⁄VBZ⁄be a⁄DT⁄a mode⁄NN⁄mode of⁄IN⁄of '
 'autobiography⁄NN⁄autobiography',
 'Those⁄DT⁄Those who⁄WP⁄who find⁄VBP⁄find ugly⁄JJ⁄ugly meanings⁄NNS⁄meaning '
 'in⁄IN⁄in beautiful⁄JJ

In [10]:
# получаем общее числу слов в тексте, но после фильтрации
len(text.words(filtrate=True))

70598

In [12]:
len(text.words(filtrate=False))

159610

In [13]:
# считаем сколько всего слов есть в корпусе по всем документам
sum(corpus.ccfs().values())

1924217

In [16]:
# объект предложения в виде экземпляра класса TaggedSentence; n это порядковый номер предложения
text.sents(0)

TaggedSentence(
	'THE⁄DT⁄The PICTURE⁄NN⁄Picture OF⁄IN⁄Of DORIAN⁄JJ⁄Dorian GRAY⁄NNP⁄Gray BY⁄IN⁄By OSCAR⁄NNP⁄Oscar WILDE⁄NNP⁄Wilde THE⁄DT⁄The PREFACE⁄NNP⁄Preface The⁄DT⁄The artist⁄NN⁄artist is⁄VBZ⁄be the⁄DT⁄the creator⁄NN⁄creator of⁄IN⁄of beautiful⁄JJ⁄beautiful things⁄NNS⁄thing',
	 n=0
)

In [9]:
# легко догадаться, что это число слов в предложении
text.sents(0).nwords

18

In [18]:
# а так мы получаем все предложения из текста не длиннее одного слова
text.sents(max_words=1)[:3]

[TaggedSentence(
 	'Harry⁄NNP⁄Harry',
 	 n=234
 ), TaggedSentence(
 	'Harry⁄NNP⁄Harry',
 	 n=281
 ), TaggedSentence(
 	'Why⁄WRB⁄Why',
 	 n=371
 )]

In [19]:
# Давайте усложним условия: не менее трех, но не более пяти слов
text.sents(min_words=3,max_words=5)[:3]

[TaggedSentence(
 	'This⁄DT⁄This is⁄VBZ⁄be a⁄DT⁄a fault⁄NN⁄fault',
 	 n=5
 ), TaggedSentence(
 	'For⁄IN⁄For these⁄DT⁄these there⁄EX⁄there is⁄VBZ⁄be hope⁄NN⁄hope',
 	 n=7
 ), TaggedSentence(
 	'That⁄DT⁄That is⁄VBZ⁄be all⁄DT⁄all',
 	 n=11
 )]

In [15]:
# А если мы хотим получить предложение в более информативном виде? Класс Token к вашим услугам.
# Метод tokens трансформирует предложение в список экземпляров этого класса.
text.sents(0).tokens(lower=True)

(Token(token='the', idx=0, pos='DT', lemma='the', nsent=0),
 Token(token='picture', idx=1, pos='NN', lemma='picture', nsent=0),
 Token(token='of', idx=2, pos='IN', lemma='of', nsent=0),
 Token(token='dorian', idx=3, pos='JJ', lemma='dorian', nsent=0),
 Token(token='gray', idx=4, pos='NNP', lemma='gray', nsent=0),
 Token(token='by', idx=5, pos='IN', lemma='by', nsent=0),
 Token(token='oscar', idx=6, pos='NNP', lemma='oscar', nsent=0),
 Token(token='wilde', idx=7, pos='NNP', lemma='wilde', nsent=0),
 Token(token='the', idx=8, pos='DT', lemma='the', nsent=0),
 Token(token='preface', idx=9, pos='NNP', lemma='preface', nsent=0),
 Token(token='the', idx=10, pos='DT', lemma='the', nsent=0),
 Token(token='artist', idx=11, pos='NN', lemma='artist', nsent=0),
 Token(token='is', idx=12, pos='VBZ', lemma='be', nsent=0),
 Token(token='the', idx=13, pos='DT', lemma='the', nsent=0),
 Token(token='creator', idx=14, pos='NN', lemma='creator', nsent=0),
 Token(token='of', idx=15, pos='IN', lemma='of', n

In [16]:
# Попробуем включить фильтрацию (это те самые rules_filter, которые мы передавали в объект Corpus, 
# но которые по умолчанию отключены параметром corpus.filtrate=False, 
# чтобы в объекте корпуса сохранялась статистика по всем словам
text.sents(0).tokens(lower=True,filtrate=True)

(Token(token='picture', idx=1, pos='NN', lemma='picture', nsent=0),
 Token(token='dorian', idx=3, pos='JJ', lemma='dorian', nsent=0),
 Token(token='gray', idx=4, pos='NNP', lemma='gray', nsent=0),
 Token(token='oscar', idx=6, pos='NNP', lemma='oscar', nsent=0),
 Token(token='wilde', idx=7, pos='NNP', lemma='wilde', nsent=0),
 Token(token='preface', idx=9, pos='NNP', lemma='preface', nsent=0),
 Token(token='artist', idx=11, pos='NN', lemma='artist', nsent=0),
 Token(token='creator', idx=14, pos='NN', lemma='creator', nsent=0),
 Token(token='beautiful', idx=16, pos='JJ', lemma='beautiful', nsent=0),
 Token(token='things', idx=17, pos='NNS', lemma='thing', nsent=0))

In [17]:
# все слова предложения с указанным номером
text.sents(0).words()

('the',
 'picture',
 'of',
 'dorian',
 'gray',
 'by',
 'oscar',
 'wilde',
 'the',
 'preface',
 'the',
 'artist',
 'is',
 'the',
 'creator',
 'of',
 'beautiful',
 'things')

In [18]:
# все леммы предложения с указанным номером
text.sents(0).lemmas()

('the',
 'picture',
 'of',
 'dorian',
 'gray',
 'by',
 'oscar',
 'wilde',
 'the',
 'preface',
 'the',
 'artist',
 'be',
 'the',
 'creator',
 'of',
 'beautiful',
 'thing')

In [19]:
# разтегированное представаление предложения
text.sents(0).untagging()

[('THE', 'DT', 'The'),
 ('PICTURE', 'NN', 'Picture'),
 ('OF', 'IN', 'Of'),
 ('DORIAN', 'JJ', 'Dorian'),
 ('GRAY', 'NNP', 'Gray'),
 ('BY', 'IN', 'By'),
 ('OSCAR', 'NNP', 'Oscar'),
 ('WILDE', 'NNP', 'Wilde'),
 ('THE', 'DT', 'The'),
 ('PREFACE', 'NNP', 'Preface'),
 ('The', 'DT', 'The'),
 ('artist', 'NN', 'artist'),
 ('is', 'VBZ', 'be'),
 ('the', 'DT', 'the'),
 ('creator', 'NN', 'creator'),
 ('of', 'IN', 'of'),
 ('beautiful', 'JJ', 'beautiful'),
 ('things', 'NNS', 'thing')]

In [10]:
# слова предложения отфильтрованные по частям речи - сущ. (в ед. и мн. числе
text.sents(0).words(pos={"NN","NNS"})

('picture', 'artist', 'creator', 'things')

In [20]:
# леммы предложения отфильтованные по частям речи - сущ. (в ед. и мн. числе)
text.sents(0).lemmas(pos={"NN","NNS"})


('picture', 'artist', 'creator', 'thing')

In [21]:
# все леммы чей postag начинается на N, то есть любые существительные
text.sents(0).lemmas(pos="N")


('picture', 'gray', 'oscar', 'wilde', 'preface', 'artist', 'creator', 'thing')

In [11]:
text.sents(0).lemmas(pos="JJ")

('dorian', 'beautiful')

In [12]:
# Предложение как есть, но уже без пунктуации, так как она была удалена на этапе очистки текста фильтрами TextCleaner
text.sents(0).raw()

'THE PICTURE OF DORIAN GRAY BY OSCAR WILDE THE PREFACE The artist is the creator of beautiful things'

In [13]:
text.words(filtrate=False)[:10]

['the',
 'picture',
 'of',
 'dorian',
 'gray',
 'by',
 'oscar',
 'wilde',
 'the',
 'preface']

In [26]:
# Можно получить частотные словари по любой части речи
verbs,cond = text.postags("VERB")[:10]
verbs

FreqDist({'be': 7086, 'have': 3158, 'say': 776, 'do': 740, 'go': 590, 'know': 518, 'come': 460, 'look': 420, 'make': 410, 'think': 390, ...})

In [27]:
adjectives,cond = text.postags("ADJ")[:10]
adjectives

FreqDist({'dorian': 582, 'good': 276, 'own': 256, 'great': 164, 'little': 164, 'young': 156, 'other': 144, 'more': 130, 'such': 130, 'old': 124, ...})

In [13]:
possessive,cond = text.postags("POS")[:10]
possessive

FreqDist({'s': 4})

In [16]:
# генерация skipgram из текста (имплементация из nltk)
for n,s in enumerate(text.skipgrams(3,2)):
    print(s)
    if n > 10:
        break

('picture', 'dorian', 'gray')
('picture', 'dorian', 'oscar')
('picture', 'dorian', 'wilde')
('picture', 'gray', 'oscar')
('picture', 'gray', 'wilde')
('picture', 'oscar', 'wilde')
('dorian', 'gray', 'oscar')
('dorian', 'gray', 'wilde')
('dorian', 'gray', 'preface')
('dorian', 'oscar', 'wilde')
('dorian', 'oscar', 'preface')
('dorian', 'wilde', 'preface')


In [18]:
# генерация ngram из текста (имплементация из nltk)
for n,s in enumerate(text.ngrams(3)):
    print(s)
    if n > 10:
        break

('picture', 'dorian', 'gray')
('dorian', 'gray', 'oscar')
('gray', 'oscar', 'wilde')
('oscar', 'wilde', 'preface')
('wilde', 'preface', 'artist')
('preface', 'artist', 'creator')
('artist', 'creator', 'beautiful')
('creator', 'beautiful', 'thing')
('beautiful', 'thing', 'reveal')
('thing', 'reveal', 'art')
('reveal', 'art', 'conceal')
('art', 'conceal', 'artist')


In [27]:
#[(n,corpus.cfs(n).get('said',0)) for n in range(corpus.ndocs)]
[(n,corpus.cfs(n,'said')) for n in range(corpus.ndocs)]

[(0, 401),
 (1, 557),
 (2, 375),
 (3, 486),
 (4, 1474),
 (5, 24),
 (6, 0),
 (7, 2),
 (8, 234),
 (9, 51),
 (10, 390),
 (11, 430),
 (12, 70),
 (13, 341),
 (14, 1286),
 (15, 567),
 (16, 356),
 (17, 1451),
 (18, 531),
 (19, 166),
 (20, 262)]

In [28]:
[(n,corpus.tfidf(n,'said')) for n in range(corpus.ndocs)]


[(0, 0.00016092431058203172),
 (1, 0.0001464246459680258),
 (2, 0.00015695153735030426),
 (3, 0.00022664684706076194),
 (4, 0.00046786045503820624),
 (5, 0.0005016983462152395),
 (6, 0),
 (7, 4.6050178545948135e-05),
 (8, 0.0002369290143741486),
 (9, 0.0001130582203935224),
 (10, 0.0006388076686500319),
 (11, 0.00041020179084672563),
 (12, 4.736252242213623e-05),
 (13, 0.0002434938236414988),
 (14, 0.00019914731966168658),
 (15, 0.00017248295113736134),
 (16, 0.0002453879949185229),
 (17, 0.0003662777417844791),
 (18, 0.000529948190193066),
 (19, 0.00013452648869904027),
 (20, 0.00016156296068542102)]

In [41]:
# набор наиболее весомых по метрике TFIDF слов в каждом тексте
res = [(n,corpus.tfidf(n,sort=-1)[0:7],
        os.path.basename(path)
 ) 
     for n,path in enumerate(corpus.filenames())
]
res[3]

(3,
 [('holmes', 0.013378570292080937),
  ('sherlock', 0.002169412950320131),
  ('watson', 0.0014991888798649405),
  ('lestrade', 0.0011004018854958346),
  ('rucastle', 0.0011004018854958346),
  ('mccarthy', 0.001071443941140681),
  ('simon', 0.0008946032784825284)],
 'doyle_the_adventures.txt')

In [27]:
# отображение слов из 3-го текста в порядковый номер в словаре tdidf
ordered_by_tfidf = {word:n for n,(word,_) in enumerate(corpus.tfidf(3,sort=-1))}

In [36]:
#ordered_by_tfidf

In [29]:
# как видим метрика хотя и дает наибольшие веса словам важным для контекста произведения - именам главных персонажей,
# но очень быстро скатывается до уровня частоупотребляемых вместе с этими словами сокращений типа mr.
# чья документтная частота очень и очень высока 
# (но их можно удалять фильтрацией, которую мы на уровне корпуса не включали, поэтому все стоп-слова и остались на месте)
corpus.dfs('mr')

16

In [30]:
# 
ordered_by_tfidf['mr'] # на  8 месте если упоядочить слова по TFIDF 

8

In [31]:
ordered_by_tfidf['said']

83

In [32]:
corpus.cfs(3,'mr')

275

In [33]:
corpus.dfs('holmes')

1

In [34]:
corpus.cfs(3,'holmes')

462

In [30]:
corpus.tfidf(6,"red")

0.00020712636290766274

In [30]:
corpus.cfs(3,'said')

486

In [31]:
corpus.dfs('said')

20

In [32]:
corpus.tfidf(3,'said')

0.00022664684706076194

In [26]:
corpus.tfidf(6,sort=-1)[:10]

[('prospero', 0.007551523202290425),
 ('courtiers', 0.0038881773578561016),
 ('waltzers', 0.0037757616011452125),
 ('mummer', 0.0037757616011452125),
 ('prince', 0.0036805490437174567),
 ('ebony', 0.0029662764061375003),
 ('musicians', 0.002413282532933419),
 ('revellers', 0.002413282532933419),
 ('suite', 0.002071538600240377),
 ('sable', 0.0020565044356389405)]

In [46]:
help(corpus.tfidf)

Help on method tfidf in module nlptk.mining.text:

tfidf(n_doc, texts: List[List[str]] = None, token=None, sort=False) method of nlptk.mining.text.Corpus instance



In [31]:
print(corpus.cfs(1,'rochester'))
print(sum(corpus.cfs(1).values()))
print(corpus.dfs('rochester'))
print(corpus.tf(1,'rochester'))
print(corpus.idf('rochester'))
print(corpus.tfidf(1,token='rochester'))


312
185598
1
0.0016810525975495426
3.044522437723423
0.005118002352232825


In [32]:
input("--cfc--")
pprint(corpus.cfs(0,sort=-1)[:10])
input("--ccfc--")
pprint(corpus.ccfs(sort=-1)[:10])
input("--dfc--")
pprint(corpus.dfs(sort=-1)[:10])
input("--tfidf--")

#for n in range(corpus.ndocs):
#    pprint(corpus.tfidf(n,sort=-1)[:10]) 
            

--cfc--
[('the', 4320),
 ('to', 4127),
 ('of', 3596),
 ('and', 3529),
 ('her', 2216),
 ('i', 2046),
 ('a', 1945),
 ('in', 1861),
 ('was', 1843),
 ('she', 1703)]
--ccfc--
[('the', 104005),
 ('and', 64431),
 ('of', 53145),
 ('to', 51249),
 ('a', 42386),
 ('i', 34454),
 ('in', 29525),
 ('he', 28953),
 ('was', 26894),
 ('it', 24706)]
--dfc--
[('over', 21),
 ('once', 21),
 ('moment', 21),
 ('a', 21),
 ('minutes', 21),
 ('by', 21),
 ('four', 21),
 ('not', 21),
 ('from', 21),
 ('loud', 21)]
--tfidf--


''

In [15]:
# общее число слов-одиночек
all_hapaxes = corpus.hapaxes()
len(all_hapaxes)

18047

In [42]:
# н-да, вот что значит не включать фильтрацию на корпусе ...
sorted(all_hapaxes)[:10]

['a-bleeding',
 'a-blowing',
 'a-buttin',
 'a-callin',
 'a-chaffin',
 'a-changing',
 'a-crossin',
 'a-doin',
 'a-doing',
 'a-done']

In [25]:
# получаем слова-одиночки для каждого текста в корпусе
hapaxes = [(path, corpus.hapaxes(n)) for n,path in enumerate(corpus.filenames())]
hapaxes[0][1][:10]

['uniting',
 'city',
 'pollution',
 'liberties',
 'sportive',
 'arrear',
 'heretofore',
 'retain',
 'bath',
 'cheap']

In [20]:
# сколько у нас хапаксов на каждый текст
[(h[0],len(h[1])) for h in hapaxes]

[('Austin Pride and Prejudice.txt', 2794),
 ('bronte_jane_txt.txt', 7459),
 ('bronte_wuthering_txt.txt', 4381),
 ('doyle_the_adventures.txt', 3814),
 ('dreiser_sister.txt', 4629),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 547),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 585),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 384),
 ('fitzgerald_great_gatsby_txt.txt', 3594),
 ('Franz Kafka - Metamorphosis.txt', 1308),
 ('John Steinbeck - Of Mice and Men.txt', 1435),
 ('kipling_jungle_book_txt.txt', 2238),
 ('london_white_txt.txt', 3232),
 ('stevenson_treasure_island_txt.txt', 3024),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 10277),
 ('stoker_dracula_txt.txt', 4515),
 ('twain_tom_sawyer_txt.txt', 3746),
 ('walter_scott_ivanhoe_txt.txt', 5859),
 ('wells_invisible_man_txt.txt', 2994),
 ('wells_war_of_the_worlds_txt.txt', 3384),
 ('wilde_picture_of_dorian_gray_txt.txt', 3585)]

In [17]:
# общекорпусная частота слова
corpus.ccfs('said')


9454

In [19]:
corpus.ccfs("am")

2083

In [13]:
# частота слова для корпуса с номером 0
corpus.cfs(0,'vicious')

1

In [18]:
corpus.cfs(9,"gregor")

298

In [17]:
corpus.ccfs("s")

7905

In [21]:
corpus.dfs('s')

21

In [24]:
pos,_ = texts[10].postags("POS")
pprint(pos)


FreqDist({})


In [23]:
corpus.cfs(9,sort=1)[:10]

[('troubled', 1),
 ('transformed', 1),
 ('vermin', 1),
 ('armour-like', 1),
 ('domed', 1),
 ('divided', 1),
 ('arches', 1),
 ('sections', 1),
 ('bedding', 1),
 ('cover', 1)]